# Modularización y Configuración Inicial de Proyecto FastAPI

Vamos a modularizar el codigo de mi API, para que sea mas profesional. Y lo camos hacer paso a paso. En las siguientes secciones, lo iremos modularizando aun mas.

![image](https://imgur.com/uPOtI3o.png)

## Paso 1

Crear la estructura de subdirectorios tal y como se muestra en la figura

## PASO 2 -- Crear archivos __init__.py:

Para que Python reconozca estos directorios como paquetes, se necesita crear un archivo vacío llamado `__init__`.py dentro de cada uno de ellos.

Cuando Python se encuentra con un directorio que contiene un archivo `__init__.py`, lo reconoce inmediatamente como un paquete y no como un directorio común. Dicho archivo puede estar vacio o no.

Ahora que tenemos la estructura, ¡empecemos con la modularización!

## Modularizacion

### Paso 1 -- Centralizar la Configuracion

Desde el archivo `config.py` se leeran el contenido del archivo `.env`, asegurate de crearlo en la raiz del proyecto(mi api).

```py
import os
from dotenv import load_dotenv

# Carga las variables de entorno desde el archivo .env
load_dotenv()

class Settings:
    """
    Clase para manejar las configuraciones de la aplicación
    """
    def conection_string(self) -> str: 
        mongo_details = os.environ["MONGO_DETAILS"]
        return mongo_details
    

# Instancia de Settings para ser importada y usada en toda la aplicación
settings = Settings()
```
Y se hacen las respectivas modificaciones en el `main.py`:

```py
# Importar la instancia de settings desde app.core.config
from core.config import settings

# --- Configuración de MongoDB ---
MONGO_DETAILS = settings.conection_string()
```
Probamos, levantando los tres servicios con `docker compose`, si es necesario volvemos a buildiar. Y funciona.

### Paso 2: Mover la Lógica de Conexión a la Base de Datos a `app/core/database.py`

Ahora vamos a encapsular la lógica de conexión y desconexión de MongoDB en un módulo dedicado. Se crear el archivo `app/core/database.py`.

Abre el archivo, y si no entiendes algo, ve el ejemplo sencillo que hay acontinuacion(Espero que estos ejemplos te ayuden a comprender mejor las anotaciones de tipo y por qué esa línea específica es la solución correcta para tu problema con `Pylance`. ¡Es una herramienta muy útil una vez que te acostumbras a ella!)

In [4]:
from typing import Dict

# Anotamos que 'edades' es un diccionario donde las claves son strings (nombres)
# y los valores son enteros (edades).
edades: Dict[str, int] = {
    "Alice": 30,
    "Bob": 24,
    "Charlie": 35
}

print(f"Diccionario de edades: {edades}")
print(f"Edad de Alice: {edades['Alice']}")

# Esto funcionaría sin problemas porque 40 es un int
edades["David"] = 40
print(f"Diccionario actualizado: {edades}")

# Esto causaría una advertencia de Pylance/MyPy porque "veinte" no es un int
edades["Eve"] = "veinte"
# print(f"Diccionario con error de tipo (comentado): {edades}")

# Esto causaría una advertencia de Pylance/MyPy porque 123 no es un str para la clave
# edades[123] = 50
# print(f"Diccionario con error de tipo en clave (comentado): {edades}")


Diccionario de edades: {'Alice': 30, 'Bob': 24, 'Charlie': 35}
Edad de Alice: 30
Diccionario actualizado: {'Alice': 30, 'Bob': 24, 'Charlie': 35, 'David': 40}


como `db_client` es ahora un diccionario, las funciones de conexion y desconexion, deben modificarse.

In [ ]:
# database.py

from motor.motor_asyncio import AsyncIOMotorClient
from fastapi import HTTPException
from typing import Optional, Dict
from core.config import settings


# Guarda la conexión global a MongoDB
db_client: Dict[str, Optional[AsyncIOMotorClient]] = {
    "client": None,
}

async def connect_to_mongo():
#    global db_client
    try:
        db_client['client'] = AsyncIOMotorClient(settings.conection_string())
        print(f'Conexion Exitosaa a la BD: {settings.conection_string()}')
    except Exception as e:
        raise HTTPException(status_code=500, detail={"message":"Error de Servidor BD no Disponib"})



async def close_mongo_connection():
    print('hola hijuemadre')
    if db_client['client']:
        db_client['client'].close()
        print('Se ha cerrado la conexion')
    else:
        print('La conexión a MongoDB no se estableció, no hay nada que cerrar.')   

de `main.py` modificamos las funciones de `startup` y `shutdown`

```py
from fastapi import FastAPI, HTTPException, Path
from fastapi.responses import HTMLResponse, JSONResponse

from datos import STUDENTS as students

# Importar la instancia de settings desde app.core.config
from core.config import settings

# Importar las funciones de conexión/desconexión de la base de datos
# Importamos 'db' para acceder al cliente
from core.database import connect_to_mongo, close_mongo_connection, db_client

# --- Configuración de MongoDB ---
MONGO_DETAILS = settings.conection_string()


# Create a FastAPI application instance
app = FastAPI()
app.title = "API Prototipo Plataforma de Comercializacion"
app.version = "0.0.3"


@app.on_event("startup")
async def startup_db_client():
    await connect_to_mongo()
    

@app.on_event("shutdown")
async def shutdown_db_client():
    await close_mongo_connection()
```